In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as rio

In [2]:
IDir = r'/Volumes/PhD/imagery/masters/output/dNBR/'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/MSM1_ROC/'

shp = r'/Volumes/PhD/imagery/masters/ROC/ROC_basins.shp'
shp = gpd.read_file(shp)


In [8]:
def MSM1(dnbr,
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = np.array([0.67])
    b_3 = 0.7
    
    # File naming
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    s = np.multiply(dnbrdata, b_2)
    
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
   #R = RI['RI']
    for val in R:
        s1 = np.multiply(s, val)
        p1 = []
        for i in s1:
            lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
            prob = (e ** lnx) / (1.0 + e ** lnx)
            p1.append(prob)
        
        # Append probability array, flatten data, reshape into image
        prob_arr = np.array(p1)
        p_final = prob_arr.flatten()
        pixarr[x] = p_final
        pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
        out_dnbr = xr.Dataset()
        out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                                coords = {'x': dnbr.coords['x'],
                                        'y': dnbr.coords['y']})
        out_dnbr = out_dnbr.where(~np.isnan(out_dnbr), np.nan)
        
        if save == True:
            outname = f"{basin}_MSM1_{int(val*4)}mmhr.tif"
            out_dnbr.rio.to_raster(os.path.join(ODir, outname), driver='GTIFF')
    

In [37]:
def MSM1_ROC(dnbr, RI,
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = np.array([0.67])
    b_3 = 0.7
    
    # File naming
    
    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
    print(np.mean(dnbrdata))
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    s = np.multiply(dnbrdata, b_2)
    
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    R = RI
    
    s1 = np.multiply(s, R)
    p1 = []
    for i in s1:
        lnx =  b + (b_1 * x_1 * R) + i + (b_3 * x_3 * R)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
    
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                    'y': dnbr.coords['y']})
    out_dnbr = out_dnbr.where(~np.isnan(out_dnbr), np.nan)
    
    if save == True:
        outname = f"{basin}_MSM1_{int(R*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(ODir, outname), driver='GTIFF')
    

In [39]:
for index, row in shp.iterrows():
    # Extract BASIN_ID and RI for each shape
    basin_id = row['BASIN_ID']
    basin = basin_id
    RI = row['RI']
    file = os.path.join(IDir, f'basin_{basin_id}_dNBR.tif')
    image = rxr.open_rasterio(file)
    MSM1_ROC(image, RI, True)
    # Print the values
    print(f"BASIN_ID: {basin_id}, RI: {RI}")

0.471854
BASIN_ID: 17499, RI: 5.08
0.07236327
BASIN_ID: 3115, RI: 7.11
0.2164977
BASIN_ID: 3589, RI: 7.11
0.692429
BASIN_ID: 5683, RI: 12.95
0.6059562
BASIN_ID: 5920, RI: 6.1
0.60217655
BASIN_ID: 7185, RI: 6.1
0.66781485
BASIN_ID: 7262, RI: 9.91
0.6439657
BASIN_ID: 10810, RI: 11.68
0.6578977
BASIN_ID: 10814, RI: 11.68
0.6815953
BASIN_ID: 10865, RI: 11.68
0.619411
BASIN_ID: 10890, RI: 11.68
0.45053804
BASIN_ID: 11112, RI: 25.15
0.62342477
BASIN_ID: 11209, RI: 9.14
0.41562474
BASIN_ID: 11514, RI: 25.15
0.64564204
BASIN_ID: 11730, RI: 9.14
0.5911507
BASIN_ID: 11857, RI: 9.14
0.45556048
BASIN_ID: 11896, RI: 25.15
0.47930628
BASIN_ID: 12057, RI: 19.56
0.522776
BASIN_ID: 12080, RI: 25.15
0.56683004
BASIN_ID: 12288, RI: 9.14
0.6698553
BASIN_ID: 12571, RI: 9.14
0.6218152
BASIN_ID: 12729, RI: 9.14
0.6925132
BASIN_ID: 12918, RI: 6.1
0.541659
BASIN_ID: 13132, RI: 25.15
0.24137852
BASIN_ID: 13281, RI: 19.56
0.58077085
BASIN_ID: 13378, RI: 9.14
0.5444781
BASIN_ID: 14179, RI: 9.14
0.51629907
BASIN_I

In [6]:
for filename in os.listdir(IDir):
    file = os.path.join(IDir, filename)
    basin = filename.split('_')
    basin = basin[1]
    print(basin)
    image = rxr.open_rasterio(file)
    MSM1_ROC(image, RI_dict, True)
    

    


10009
Got dNBR
calculated S
0      17499
1       3115
2       3589
3       5683
4       5920
       ...  
732    15065
733    35852
734    36553
735    36955
736    37056
Name: BASIN_ID, Length: 737, dtype: int64 RI


ValueError: operands could not be broadcast together with shapes (75,) (737,) 

In [7]:
for filename in os.listdir(IDir):
    file = os.path.join(IDir, filename)
    basin = filename.split('_')
    basin = basin[1]
    print(basin)
    image = rxr.open_rasterio(file)
    MSM1(image, False)
    break

10009


NameError: name 'dnbrdata' is not defined